In [2]:
# This notebook analyzes MAANG stocks from the perspective of the typical
# Black-Scholes model using day-over-day returns.

# Using daily returns has the benefit of potentially reducing noise associated
# with seasonality etc, but more likely the lookback window used by market makers
# for example will be much shorter, say 30 days. One downside is that the
# approximation associated with Geometric Brownian Motion is less accurate.
# Intra-day (high-frequency) returns may be preferred for this reason in practice.

Bad pipe message: %s [b'\x16\xc9\xa4\xb8\x02\x97f|\r\x896M.\xcbD=\xd0/ \xbe\xc9\x91Q\xa0\xd6`\xe7\xd4\xaa\x8c\xbe\x0f\xd4,\xa0{\xb28\xb0\xc2\x04\x97?4x\xb2|\xb5\x07\xa8\xd8\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17']
Bad pipe message: %s [b"`%\xcd\xf9\x06\xcf\x1c\x0eB\x06\xedpS\x05l\r\x85\xbb\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0"]
Bad pipe message: %s [b'\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00']
Bad pipe message: %s [b"\xd0o9\x05\x8a\xa3\x08